In [1]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.2/859.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.5 MB/s eta 0:00:00


In [2]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 10.3 MB/s eta 0:00:00


##Core Components: The Tool Functions

In [3]:
# @title Tool Functions
# These functions simulate calling deployed ML models.

def fetch_movie_data(title: str) -> dict:
    """
    Fetches movie data from a live source like the TMDb API[cite: 6].
    """
    print(f"TOOL: Fetching data for '{title}'...")
    # Dummy data for demonstration
    return {
        "title": title,
        "budget": 160000000, # [cite: 2]
        "plot_overview": "A thief who steals corporate secrets through use of dream-sharing technology...", # [cite: 2]
    }

def predict_revenue(movie_data: dict) -> float:
    """Calls the deployed regression model to predict box office revenue."""
    print("TOOL: Predicting revenue...")
    return 829895144.0

def classify_success(movie_data: dict) -> dict:
    """Calls the deployed classification model to predict success."""
    print("TOOL: Classifying success...")
    return {"class": "High Success", "confidence": 0.92}

def identify_genres_from_plot(plot_overview: str) -> list:
    """Calls the deployed NLP model to identify genres from a plot summary."""
    print("TOOL: Identifying genres from plot...")
    return ["Action", "Science Fiction", "Thriller"]

def get_recommendations(plot_overview: str, genres: list) -> list:
    """Uses content-based filtering to find similar movies."""
    print("TOOL: Generating recommendations...")
    return [
        {"title": "The Matrix", "imdb_id": "tt0133093"},
        {"title": "Shutter Island", "imdb_id": "tt1130884"},
    ]

## Implementation with LangGraph

### Step 1: Define the Graph State

In [4]:
# @title Define LangGraph State and Nodes
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END

# Define the state object that will be passed between nodes
class GraphState(TypedDict):
    """Represents the state of our graph."""
    movie_title: str
    movie_data: Dict
    predicted_revenue: float
    success_metrics: Dict
    identified_genres: List[str]
    recommendations: List[Dict]
    final_result: Dict


### Step 2: Create Graph Nodes

In [5]:
# Define the nodes of the graph
def data_fetcher_node(state: GraphState) -> GraphState:
    title = state["movie_title"]
    movie_data = fetch_movie_data(title)
    return {"movie_data": movie_data}

def revenue_node(state: GraphState) -> GraphState:
    revenue = predict_revenue(state["movie_data"])
    return {"predicted_revenue": revenue}

def success_node(state: GraphState) -> GraphState:
    success = classify_success(state["movie_data"])
    return {"success_metrics": success}

def genre_node(state: GraphState) -> GraphState:
    genres = identify_genres_from_plot(state["movie_data"]["plot_overview"])
    return {"identified_genres": genres}

def recommendation_node(state: GraphState) -> GraphState:
    recs = get_recommendations(
        state["movie_data"]["plot_overview"],
        state["identified_genres"]
    )
    return {"recommendations": recs}

def compile_result_node(state: GraphState) -> GraphState:
    result = {
        "movie_title": state["movie_title"],
        "analysis": {
            "predicted_revenue": state["predicted_revenue"],
            "success_prediction": state["success_metrics"],
            "identified_genres": state["identified_genres"],
            "recommendations": state["recommendations"]
        }
    }
    print("\n--- FINAL RESULT COMPILED ---")
    return {"final_result": result}

### Step 3: Wire the Graph

In [6]:
# @title Wire and Run the LangGraph Workflow
import json

# Initialize the stateful graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("fetch_data", data_fetcher_node)
workflow.add_node("predict_revenue", revenue_node)
workflow.add_node("classify_success", success_node)
workflow.add_node("identify_genres", genre_node)
workflow.add_node("get_recommendations", recommendation_node)
workflow.add_node("compile_results", compile_result_node)

# Define the workflow edges
workflow.set_entry_point("fetch_data")
workflow.add_edge("fetch_data", "predict_revenue")
workflow.add_edge("fetch_data", "classify_success")
workflow.add_edge("fetch_data", "identify_genres")
workflow.add_edge(["predict_revenue", "classify_success", "identify_genres"], "get_recommendations")
workflow.add_edge("get_recommendations", "compile_results")
workflow.add_edge("compile_results", END)

# Compile and run the graph
app = workflow.compile()
inputs = {"movie_title": "Inception"}
result = app.invoke(inputs)

print(json.dumps(result['final_result'], indent=2))

TOOL: Fetching data for 'Inception'...
TOOL: Identifying genres from plot...
TOOL: Classifying success...
TOOL: Predicting revenue...
TOOL: Generating recommendations...

--- FINAL RESULT COMPILED ---
{
  "movie_title": "Inception",
  "analysis": {
    "predicted_revenue": 829895144.0,
    "success_prediction": {
      "class": "High Success",
      "confidence": 0.92
    },
    "identified_genres": [
      "Action",
      "Science Fiction",
      "Thriller"
    ],
    "recommendations": [
      {
        "title": "The Matrix",
        "imdb_id": "tt0133093"
      },
      {
        "title": "Shutter Island",
        "imdb_id": "tt1130884"
      }
    ]
  }
}
